<a href="https://colab.research.google.com/github/scliu7/useful_stuff/blob/main/active_matters/FRAP/FRAP_high_pH_MT_aging_10142022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os

from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

import bokeh.io
from bokeh.plotting import figure, show


from bokeh.io import output_notebook
output_notebook()

!pip install iqplot --quiet
import iqplot

In [ ]:
# Create an empty dataframe
df = pd.DataFrame(index=range(200),columns=range(0))

In [ ]:
folder_path = '/content/drive/MyDrive/FRAP/101422/same_ph'
for i in range(len(os.listdir(folder_path))):
  x = i + 1
  file_path = os.path.join(folder_path, "R" + str(x) + ".xlsx")
  col = "reaction_" + str(x)
  df[col] = pd.read_excel(file_path, nrows = 199)
  #col.append("reaction_" + str(x))

normalized_df=(df-df.min())/(df.max()-df.min())


In [ ]:
cols = len(normalized_df.axes[1])
p2 = figure(plot_width=600, plot_height=300, 
            title="Fluorescence recovery after photobleaching", 
            x_axis_label='frames', 
            y_axis_label='normalized intensity')

for i in [1, 2]:
  x = i + 1
  col = "reaction_" + str(i + 1)
  x = list(range(1, len(normalized_df[col].dropna())))
  p2.line(x= x ,
        y = normalized_df[col].dropna(),
        #source=source,
        legend_label = col,
        color = Category10[10][i])

show(p2)


Equation from Daniel Axelrod paper. EQ 19

$$
  D = (w^2 /\, 4\tau_{1/2})\, \gamma_D
$$

Where
$$ \gamma_D = 0.88 $$
for circular beams, and
$$ w $$
is radius of the uniform laser beam

The optical setup we used has a resolution of 0.294 um/pixel

In [ ]:
def find_closest(df, col_name):
  value = (df[col_name].max() - df[col_name].min()) / 2
  dist = (df[col_name] - value).abs()
  return dist.idxmin()

In [ ]:
# Calculate the tau half for mid point
tau_half = []
for i in range(cols):
  col = "reaction_" + str(i + 1)
  tau_half.append(find_closest(normalized_df, col) * 30)
tau_half

[330, 270, 480]

In [ ]:
res = 0.294
r_circle = 390
w = res * r_circle
D = []

for num in tau_half:
  D.append(0.88 * w ** 2 / (4 * num))


In [ ]:
show(p2)
#print("For Reaction 1 (high pH, and 5 min MT aging), the diffusion coefficient of MT is", "%0.2f" % D[0], "um^2/s")
print("For Reaction 2 (high pH, and 120 min MT aging in tube), the diffusion coefficient of MT is", "%0.2f" % D[1], "um^2/s")
print("For Reaction 3 (high pH, and 120 min MT aging on chip), the diffusion coefficient of MT is", "%0.2f" % D[2], "um^2/s")


For Reaction 2 (high pH, and 120 min MT aging in tube), the diffusion coefficient of MT is 10.71 um^2/s
For Reaction 3 (high pH, and 120 min MT aging on chip), the diffusion coefficient of MT is 6.03 um^2/s
